In [15]:
# Import libraries
import pgmpy.models
import pgmpy.inference
import numpy as np
from enum import Enum

In [27]:
numSlices = 24

class NodeNames(Enum):
    MITM = "MITM"
    SRM = "SRM"
    UC = "UC"
    MC = "MC"
    CC = "CC"

In [30]:
# Create a dynamic bayesian network
model = pgmpy.models.DynamicBayesianNetwork()
# Add nodes
model.add_nodes_from([NodeNames.MITM.value, NodeNames.SRM.value, NodeNames.UC.value, NodeNames.UPS.value])
# Add edges
for i in range(0, numSlices):
    if i != (numSlices - 1):
        for nodeName in NodeNames:
            model.add_edges_from([((nodeName.value, i), (nodeName.value, i+1))])
    
    model.add_edges_from([((NodeNames.MITM.value, i), (NodeNames.SRM.value, i)), ((NodeNames.MITM.value, i), (NodeNames.UC.value, i)),
                          ((NodeNames.SRM.value, i), (NodeNames.UPS.value, i)), ((NodeNames.UC.value, i), (NodeNames.UPS.value, i))])

# Print edges
print('--- Edges ---')
print(model.edges())
print()
            

--- Edges ---
[(<DynamicNode(MITM, 0) at 0x7f188b18b040>, <DynamicNode(MITM, 1) at 0x7f188b18beb0>), (<DynamicNode(MITM, 0) at 0x7f188b18b040>, <DynamicNode(SRM, 0) at 0x7f188b126b50>), (<DynamicNode(MITM, 0) at 0x7f188b18b040>, <DynamicNode(UC, 0) at 0x7f188b126490>), (<DynamicNode(SRM, 0) at 0x7f188b18bee0>, <DynamicNode(SRM, 1) at 0x7f188b126970>), (<DynamicNode(SRM, 0) at 0x7f188b18bee0>, <DynamicNode(UPS, 0) at 0x7f188b126910>), (<DynamicNode(UC, 0) at 0x7f188b18b190>, <DynamicNode(UC, 1) at 0x7f188b1260d0>), (<DynamicNode(UC, 0) at 0x7f188b18b190>, <DynamicNode(UPS, 0) at 0x7f188b15a640>), (<DynamicNode(UPS, 0) at 0x7f188b18b130>, <DynamicNode(UPS, 1) at 0x7f188b126bb0>), (<DynamicNode(MITM, 1) at 0x7f188b18beb0>, <DynamicNode(SRM, 1) at 0x7f188b1267c0>), (<DynamicNode(MITM, 1) at 0x7f188b18beb0>, <DynamicNode(UC, 1) at 0x7f188b126e50>), (<DynamicNode(SRM, 1) at 0x7f188b126970>, <DynamicNode(UPS, 1) at 0x7f188b126ca0>), (<DynamicNode(UC, 1) at 0x7f188b1260d0>, <DynamicNode(UPS, 1